In [1]:
!ls

BERT.ipynb                LSTM_model.hdf5.zip       demo.ipynb
CNN_model.hdf5            NN.ipynb                  finally.HDF5
GRU_model.hdf5            Untitled.ipynb            john_playing_around.ipynb
LSTM_model.hdf5           WordClouds.ipynb


In [2]:
%%capture
pip install pandas numpy nltk keras tensorflow

In [3]:
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import ssl
import requests
ssl._create_default_https_context = ssl._create_unverified_context
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from tensorflow.keras.utils import pad_sequences

2022-12-12 22:15:57.166548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


# Data Processing and Model Loading

In [4]:
data = pd.read_json("../Sarcasm_Headlines_Dataset_v2.json",lines=True)
stwrds = set(stopwords.words('english'))
data=data.drop(data[data['headline'].duplicated()].index,axis=0)

In [5]:
indx = 2002
print(data.iloc[indx])
print("\n")
print(data.iloc[indx]['headline'])

is_sarcastic                                                    1
headline        scientists successfully create artificial plac...
article_link    https://www.theonion.com/scientists-successful...
Name: 2003, dtype: object


scientists successfully create artificial placenta that tastes just as delicious as real one


In [6]:
def clean_headlines(headline):
    headline = headline.lower()
    headline_split = headline.split()
    cleaned_headline = []
    for word in headline_split:
        if word not in stwrds:
            cleaned_headline.append(word)
    
    cleaned_line = " ".join(cleaned_headline)
    return cleaned_line
data['headline'].apply(clean_headlines)

0        thirtysomething scientists unveil doomsday clo...
1        dem rep. totally nails congress falling short ...
2             eat veggies: 9 deliciously different recipes
3             inclement weather prevents liar getting work
4        mother comes pretty close using word 'streamin...
                               ...                        
28614               jews celebrate rosh hashasha something
28615    internal affairs investigator disappointed con...
28616    beautiful acceptance speech week came queer ko...
28617    mars probe destroyed orbiting spielberg-gates ...
28618                              dad clarifies food stop
Name: headline, Length: 28503, dtype: object

In [7]:
vocab_size = 20000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(data['headline'])

In [9]:
cnn_model = load_model('CNN_model.hdf5')
gru_model = load_model('GRU_model.hdf5')
lstm_model = load_model('LSTM_model.hdf5')

In [10]:
preprepped_strs = ["John only spells some words incorrectly", \
                   "professor announces two final exams students rejoice", \
                   "professor announces free 100s on the final exam for everybody", \
                  "calfornia is expericing a record warm winter becuase of global warming",\
                  "ford develops new suv that runs purely on gasoline", \
                  "scientists successfully create artificial placenta that tastes just as delicious as real one", \
                  "professor molontay announces final exam grades rate of happiness at AIT declines"]


# Demo Time Give Us Sarcastic Statements (Think news headline length and content)

In [11]:
def test_str_against_model(demo_string: list):
    demo_string[0] = demo_string[0].lower()
    demo_string_lexed = tokenizer.texts_to_sequences(demo_string)
    demo_string_pad = pad_sequences(demo_string_lexed, maxlen=150, padding='post', truncating='post')
    results = []
    results.append((cnn_model.predict(demo_string_pad).flatten(), "CNN"))
    results.append((gru_model.predict(demo_string_pad).flatten(), "GRU"))
    results.append((lstm_model.predict(demo_string_pad).flatten(), "LSTM"))
    predicts = []
    print(results)
    for pred in results:
        if pred[0] < 0.5:
            predicts.append(("Not Sarcastic", pred[1]))
        else:
            predicts.append(("Sarcastic", pred[1]))
            
    return predicts

In [12]:
test_str_against_model([""])



1/1 [==============================] - 1s 545ms/step
[(array([0.36877713], dtype=float32), 'CNN'), (array([0.6071612], dtype=float32), 'GRU'), (array([0.40747514], dtype=float32), 'LSTM')]


[('Not Sarcastic', 'CNN'), ('Sarcastic', 'GRU'), ('Not Sarcastic', 'LSTM')]